In [1]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import pickle
from mpl_toolkits.mplot3d import Axes3D
from sklearn import preprocessing as pre
from scipy import ndimage, interpolate
from copy import deepcopy
%matplotlib qt

## Parameters for the simulation:

In [34]:
# Constants:
rad_per_deg = np.pi/180.
hc = 1.239847


# Exciton
Ex = 2.013  # eV
dEx = 22e-3 # linewidth eV

# Max E
max_E = 2.08

# Photon dispersion shift from bare vs device with monolayer (probably due to the index of the monolayer)
delta_E = 0.015

## Import Simulation Results

In [35]:
with open('python_RWCA_0.35.pkl', 'rb') as handle:
    results = pickle.load(handle)
    
# Ignore excess lines due to different modes
ind = np.argmin((results['E'] - max_E)**2)
results['R_no_monolayer'] = results['R_no_monolayer'][ind:]
results['R_with_monolayer'] = results['R_with_monolayer'][ind:]
results['E'] = results['E'][ind:]


### Plot Results:

In [36]:
# plt.pcolor(thetas/rad_per_deg, hc/wl[:-1], np.diff(results,axis=0))
# plt.pcolor(thetas/rad_per_deg, hc/wl, ndimage.sobel(results,axis=0)+ndimage.sobel(results,axis=0))
plt.subplot(1,2,1)
# plt.pcolor(thetas/rad_per_deg, hc/wl, results>0.006)
plt.pcolor(results['theta']/rad_per_deg, results['E'], ndimage.sobel(results['R_with_monolayer'],axis=0))
# plt.colorbar()

plt.clim([0.002,0.005])
# plt.ylim([1.96,2.09])
plt.xlabel('theta (deg)')
plt.ylabel('E (eV)')
plt.title('With Monolayer')

plt.subplot(1,2,2)
plt.pcolor(results['theta']/rad_per_deg, results['E'], results['R_no_monolayer'])
# plt.colorbar()
plt.clim([0.002,0.5])
# plt.ylim([1.96,2.09])
plt.xlabel('theta (deg)')
plt.ylabel('E (eV)')
plt.title('No Monolayer')


Text(0.5, 1.0, 'No Monolayer')

## Interpolate Photon mode
By finding the maximum intensity for each angle and then interpolating

In [37]:
# Find photon mode
plt.pcolor(results['theta'], results['E'], results['R_no_monolayer'])
x = np.zeros(results['R_no_monolayer'].shape[0])
for i in range(results['R_no_monolayer'].shape[0]):
    x[i] = results['theta'][np.argmax(results['R_no_monolayer'][i,:])]
    
photon_dispersion_line = interpolate.interp1d(x, results['E']-delta_E, fill_value='extrapolate')
plt.plot(results['theta'], photon_dispersion_line(results['theta']), 'r', alpha = 0.2)

## Define Functions for Fitting:
1. **polariton_dispersion_function**: Calculate dispersion separately for upper and lower polariton and combine them to get the overall 2D dispersion function
2. **polariton_dispersion_line**: Returns the lines of the lower and upper polaritons. 

In [38]:
# split_ind = np.argmin((results['E']-Ex)**2)
def polariton_dispersion_function(xy, g, dE, a):
    x, y = xy
    y_low = y[y<Ex]
    y_up = y[y>=Ex]
    E_lowpolariton, E_uppolariton = polariton_dispersion_line((x,y), g, dE, a)
    func1 = a*np.exp(-((y_low-E_lowpolariton)/dE)**2)/(np.sqrt(2*dE))
    func2 = a*np.exp(-((y_up-E_uppolariton)/dE)**2)/(np.sqrt(2*dE))
    return np.concatenate((func2, func1), axis=0)

def polariton_dispersion_line(xy, g, dE, a):
    x, y = xy
    E_photon = photon_dispersion_line(x)
    E_lowpolariton = 0.5*(E_photon[y<Ex] + Ex - np.sqrt((E_photon[y<Ex] - Ex)**2 + 4*g**2))
    E_uppolariton = 0.5*(E_photon[y>=Ex] + Ex + np.sqrt((E_photon[y>=Ex]- Ex)**2 + 4*g**2))
    E_polariton = np.concatenate((E_lowpolariton, E_uppolariton), axis=0)
    return E_lowpolariton, E_uppolariton


In [39]:
x = results['theta']
y= results['E']
X, Y = np.meshgrid(x,y)
z = deepcopy(results['R_with_monolayer'])
for i in range(z.shape[0]):
    z[i,:]/= z[i,:].max()
# z = ndimage.gaussian_filter(z, sigma=5)
polariton_opt, pcov = curve_fit(polariton_dispersion_function,
                                np.vstack((X.ravel(), Y.ravel())), z.ravel(),
                                p0=[0.015,0.001,0.1],
                                bounds=((0.01, 0.001, 0.),(0.02, 0.005,np.inf))
                               ) 
print(polariton_opt)
print(np.sqrt(np.diag(pcov)))

[0.0184115  0.005      0.02786764]
[1.08373855e-04 8.46390771e-05 3.19186964e-04]


In [48]:
# polariton_opt = [0.01,0.01,0.08032468]
x_deg = x/rad_per_deg
X_deg = X/rad_per_deg
plt.subplot(1,2,2)
plt.pcolor(x_deg, y, polariton_dispersion_function((X,Y),*polariton_opt).reshape(len(y),len(x)))
plt.plot(x_deg, np.ones_like(x)*Ex, '--r')
plt.plot(x_deg, photon_dispersion_line(x), 'r')
plt.ylim([y.min(),y.max()])
plt.title('Fit')

plt.subplot(1,2,1)
# plt.pcolor(x,y,z)
plt.pcolor(x_deg,y,ndimage.sobel(results['R_with_monolayer'],axis=0))
plt.clim([0.002,0.005])
# plt.ylim([1.96,2.09])
plt.xlabel('theta (deg)')
plt.ylabel('E (eV)')

l, u =  polariton_dispersion_line((X.ravel(),Y.ravel()),*polariton_opt)
plt.plot(X_deg.ravel(), np.concatenate((u,l), axis=0),'.', markersize=0.5, color='orange')
# plt.plot(results['theta'], upperpolariton_dispersion_line((results['theta'],[]), *polariton_opt))
plt.plot(x_deg, np.ones_like(x)*Ex, '--r')
# plt.plot(x, photon_dispersion_line((x, y.ravel()), *popt), 'r')
plt.plot(x_deg, photon_dispersion_line(x), 'r')
plt.title('Simulation')
plt.ylim([y.min(),y.max()])

(1.7999999999999998, 2.0795472492761253)

In [ ]:
x_test = np.zeros_like(y)
for i in range(len(x_test)):
    x_test[i] = x[np.argmax(z[i,:])]
    
plt.pcolor(x,y,z)
plt.plot(x_test, y, 'r', alpha = 0.2)

In [ ]:
x.shape